In [ ]:
import threading
from etl.etl_run import ETLRun

from models import ParentPair, ChildPair

from sqlalchemy import create_engine
from db_constants import Base
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)

In [ ]:
stop_event = threading.Event()
data_loader = DataLoader()
etl = ETLRun(stop_event, pairs_limit=25, data_loader=data_loader)
processor_thread = threading.Thread(target=data_loader.process_items)
etl.start_threads()
processor_thread.start()

try:
    # Keep the main thread running
    while True:
        pass
except KeyboardInterrupt:
    # On KeyboardInterrupt, stop the WebSocketThread
    stop_event.set()
    etl.stop_threads()
    data_loader.stop_queue_process()
    processor_thread.join()


In [ ]:

parent_pairs = etl.extractor.pair_sockets
for parent_pair, sub_pairs in parent_pairs.items():
    for sub_pair, thread in sub_pairs.items():
        print(thread.updates)